In [9]:
import pandas as pd
import re
import numpy as np
import warnings
warnings.filterwarnings('ignore')

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [10]:
df=pd.read_csv('./data/kurly_classify.csv',encoding='utf-8')
df

,category,brand,item,allergy,ingredient,classify
0,국/반찬/메인 요리,풀무원,국산콩 나또 3종,"국산콩생나또,대두,밀,쇠고기,검은콩생나또,대두,밀,쇠고기,국산콩와사비나또,대두,밀,...","고등어,아황산류,메밀,옥수수유옥수수,복숭아,오징어,홍합포함,설탕,조개류,전복,대두,...",general
1,국/반찬/메인 요리,kurly,전통 시장 물떡 (2팩),"밀,대두,우유,계란,우유,메밀,땅콩,고등어,게,새우,돼지고기,복숭아,토마토,아황산류...","조개류,정제소금,주정,떡류,조기,소스소스,참치,비타민,가공소금,오징어,구연산,복합조...",pesco
2,국/반찬/메인 요리,모두의맛집,소문난 원조 조방낙지 낙곱새,"대두,밀,새우,쇠고기,조개류,굴,난류,우유,메밀,땅콩,고등어,게,돼지고기,복숭아,토...","마늘,조개류,전복,내두유,콩기름,홍합포함",pesco
3,국/반찬/메인 요리,치즈룸x테이스팅룸,전복 솥밥 리조또,"전복미역밥,우유,조개류,전복,전복내장크림소스,대두,밀,우유,계란,닭고기,조개류,굴,...","마늘,고등어,호두,유청,소고기,껏일,포도당,구연산,가공이크림,토코페롤,변성전분,주정...",general
4,국/반찬/메인 요리,피코크,금돼지식당 통삽겹김치찌개,"대두,밀,돼지고기,쇠고기,알류,우유,메밀,땅콩,고등어,게,새우,복숭아,토마토,아황산...","마늘,고등어,향미증진제,콜산,호두,복숭아,홍합포함,인산이전분,닭고기,고춧가루,진한양...",general
...,...,...,...,...,...,...
2925,간식/과자/떡,라라스윗,발렌타인 기프트 세트,"우유,밀,계란,대두,땅콩,호두,메밀,복숭아,토마토,돼지고기","알룰로스,당류",vegan
2926,간식/과자/떡,발스낵,그랜드 감자칩 2묶음 6종 (택1),"씨솔트,와사비,칠리앤드라임,사우어크림앤드어니언,밀,우유,샐러리,겨자,사워크림앤허브,...","글루,정제소금,오일,고추냉이,정제수정제소금,감자플레이크,리브뉴클레오티드이나트롬,감자...",vegan
2927,간식/과자/떡,리터 스포트,미니 초콜릿믹스 9P,없음,"준초콜릿,옥수수",vegan
2928,간식/과자/떡,청우,아홉 번 구워만든 죽염캔디 38g,"땅콩,대두,우유,복숭아","무엇,자일리톨,박하유",vegan


In [11]:
df['ingredient']=df['ingredient'].apply(lambda x: x.replace(',',' '))
count_vect=CountVectorizer(min_df=0,ngram_range=(1,2))
ingredient_mat=count_vect.fit_transform(df['ingredient'])
print(ingredient_mat.shape)

(2930, 20982)


In [12]:
ingredient_sim=cosine_similarity(ingredient_mat,ingredient_mat)
print(ingredient_sim.shape)
print(ingredient_sim)

(2930, 2930)
[[1.         0.08804509 0.1839202  ... 0.         0.         0.0919601 ]
 [0.08804509 1.         0.05802589 ... 0.         0.         0.        ]
 [0.1839202  0.05802589 1.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 1.         0.         0.        ]
 [0.         0.         0.         ... 0.         1.         0.        ]
 [0.0919601  0.         0.         ... 0.         0.         1.        ]]


In [13]:
ingredient_sim_sorted_ind=ingredient_sim.argsort()[:,::-1]
print(ingredient_sim_sorted_ind[:1])

[[   0  453  561 ... 1776 1775  743]]


In [14]:
def find_sim_ingredient(df,sorted_ind,product_name,top_n=10,cate=0,vegan=0):
    
    title_product=df[df['item']==product_name]
    title_index=title_product.index.values

    similar_indexes=sorted_ind[title_index]
    similar_indexes=similar_indexes[similar_indexes!=title_index]

    similar_indexes=similar_indexes.reshape(-1)
    option_lst=similar_indexes

    # 카테고리 옵션
    if cate==1:
        option_lst=[]
        category=df.loc[df['item']==product_name,'category'].values
        cate_indexes=df.loc[df['category']==category[0]].index.values
    
    # 비건단계 옵션1 -같은 종류의 단계만 추천
    if vegan==1:
        option_lst=[]
        vegan_level=df.loc[df['item']==product_name,'classify'].values
        same_vegan_indexes=df.loc[df['classify']==vegan_level[0]].index.values

    # 비건단계 옵션2 -비건단계당 2개씩 추천
    elif vegan==2:
        option_lst=[]
        vegan_check={'pesco':0,'lacto_ovo':0,'lacto':0,'ovo':0,'vegan':0,'honey':0}
        cnt=0
            
    # 비건단계 옵션3 - 비건 제품만 추천
    elif vegan==3:
        option_lst=[]
        vegan_indexes=df.loc[df['classify']=='vegan'].index.values

    # 옵션 적용
    if cate==1 and vegan==0: # 카테고리
        for idx in similar_indexes:
            if idx in cate_indexes:
                option_lst.append(idx)
    
    elif cate==0 and vegan==1: # 같은비건단계
        for idx in similar_indexes:
            if idx in same_vegan_indexes:
                option_lst.append(idx)

    elif cate==0 and vegan==2: # 비건단계별
        for idx in similar_indexes:
            vegan_level=df.iloc[idx,-1]
            if vegan_level!='general' and vegan_check[vegan_level]<2:
                vegan_check[vegan_level]+=1
                option_lst.append(idx)
                cnt+=1
            elif cnt==12:
                break
            else:
                continue
    
    elif cate==0 and vegan==3: # 비건제품만
        for idx in similar_indexes:
            if idx in vegan_indexes:
                option_lst.append(idx)

    elif cate==1 and vegan==1: # 카테고리, 같은비건단계
        for idx in similar_indexes:
            if idx in cate_indexes and idx in same_vegan_indexes:
                option_lst.append(idx)

    elif cate==1 and vegan==2: # 카테고리, 비건단계별
        for idx in similar_indexes:
            if idx in cate_indexes:
                vegan_level=df.iloc[idx,-1]
                if vegan_level!='general' and vegan_check[vegan_level]<2:
                    vegan_check[vegan_level]+=1
                    option_lst.append(idx)
                    cnt+=1
                elif cnt==12:
                    break
                else:
                    continue

    elif cate==1 and vegan==3:  # 카테고리, 비건제품만
        for idx in similar_indexes:
            if idx in cate_indexes and idx in vegan_indexes:
                option_lst.append(idx)

    return df.iloc[option_lst][:top_n]

In [15]:
df.iloc[[10]]

,category,brand,item,allergy,ingredient,classify
10,국/반찬/메인 요리,비비고,총각김치1.5KG,"새우,대두,밀,조개류,굴,전복,홍합포함,잣",마늘 조개류 식염 고추 절임알타리무 배퓨레 감칠맛액젓 고초가루 하선정남해안명품멸치액...,pesco


In [22]:
find_sim_ingredient(df,ingredient_sim_sorted_ind,df.loc[10,'item'],10,1,2)

,category,brand,item,allergy,ingredient,classify
172,국/반찬/메인 요리,비비고,총각 김치,"새우,대두",마늘 조개류 식염 고추 대두함유 절임알타리무 배퓨레 감칠맛액젓 고초가루 멸치 김치 총각무,pesco
303,국/반찬/메인 요리,비비고,포기 배추김치 4.9kg,"밀,조개류,굴,전복,홍합포함,잣,새우,대두",절임배추 배추 이기지간마늘 마늘 조개류 식염 고추 쪽파 고초가루 배퓨레 채소 새우액...,pesco
1925,국/반찬/메인 요리,조선호텔김치,갈치석박지,"새우,우유,대두,쇠고기,조개류,전복,밀,잣",마늘 식염 쪽파 고추 정제수 생강 아스파탐 감미료 열무,vegan
1935,국/반찬/메인 요리,조선호텔김치,조선 주니어 배추김치 400g,"새우,우유,대두,쇠고기,조개류,전복,밀,잣",마늘 상강 식염 사과 삽블가루 고춧가루 부추 잡쓸플 새우젯우 설탕올리고당 양파,vegan
1120,국/반찬/메인 요리,스마일찬,진미채 간장볶음,"대두,밀,오징어,계란,쇠고기,고등어,게,새우,돼지고기,호두,닭고기,조개류",마늘 조미건어포류 할지대도 계란,ovo
1121,국/반찬/메인 요리,해녀의부엌x더반찬,제주뿔소라 미역국,"우유,메밀,대두,밀,게,새우,조개류,소라,전복,홍합,알류,땅콩,고등어,돼지고기,복숭...",대파 미역 토마토 복숭아 전복소스별칭 유지방 양파,lacto
1115,국/반찬/메인 요리,비움반찬,미역줄기볶음,"대두,닭고기,알류,우유,메밀,땅콩,밀,고등어,게,새우,돼지고기,복숭아,토마토,아황산...",우유 대두유 징어 대파 다진 열장미역풀기 미역 메밀 규소수지 양파,lacto
1116,국/반찬/메인 요리,빕스,고르곤졸라 피자,"밀,대두,우유,쇠고기,계란,새우,땅콩,호두,닭고기,복숭아,오징어,고등어,메밀,아황산...",빵류 피자치즈 마요네즈,lacto_ovo
1123,국/반찬/메인 요리,콜린스 다이닝,브리또볼,"달걀,우유,대두,밀,돼지고기,토마토,닭고기,쇠고기,조개류,바지락,모시조개,메밀,땅콩...",버터밀크 팡이 정제수 식물성유지 난황 난황액 기타가공품 일가로 절임식품 우유 소금 ...,lacto_ovo
1117,국/반찬/메인 요리,맑은물에,참좋은 국산콩 순두부 2개입,"대두,우유,계란,밀,땅콩",혼합제제 구연산삼나트륨 계란,ovo
